In [1]:
import nocd
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix
import pandas as pd

import dgl
import numpy as np
import torch as th
from dgl.nn import EdgeGATConv
from dgl.nn import GATConv

%matplotlib inline

# torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [70]:
print(dgl.__version__)

2.0.0+cu121


In [2]:
import pickle
with open('subgraph.pkl', 'rb') as file:
    subgraph = pickle.load(file)

In [3]:
subgraph.ndata['feature']=subgraph.ndata['feature'].to(torch.float)
subgraph.edata['feature']=subgraph.edata['feature'].to(torch.float)

In [4]:
class EdgeGATNet(nn.Module):
    def __init__(self, in_node_feats, in_edge_feats, hidden_feats,out_feats):
        super(EdgeGATNet, self).__init__()
        self.edge_gat_conv1 = EdgeGATConv(in_node_feats, in_edge_feats,hidden_feats,1,allow_zero_in_degree=True)
        self.edge_gat_conv2 = GATConv(hidden_feats, out_feats,1,allow_zero_in_degree=True)

    def forward(self, g, node_feats, edge_feats):
        # Update node and edge representations using EGAT layers
        node_feats= self.edge_gat_conv1(g, node_feats, edge_feats)
        node_feats= F.relu(node_feats)
        node_feats= self.edge_gat_conv2(g, node_feats)

        return node_feats

In [5]:
in_node_feats = subgraph.ndata['feature'].shape[1]
in_edge_feats = subgraph.edata['feature'].shape[1]
hidden_feats = 64  # You can adjust this based on your graph and task
out_feats = 2  # Number of communities you want to detect

In [6]:
# Initialize the model
model = EdgeGATNet(in_node_feats, in_edge_feats, hidden_feats, out_feats)

# Assuming you have a data loader for your graph
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [7]:
def deep_graph_infomax_loss(embeddings, pos_mask, neg_mask):
    # Compute dot product between positive and negative samples
    pos_score = torch.sum(embeddings * pos_mask, dim=-1)
    neg_score = torch.sum(embeddings * neg_mask, dim=-1)

    # Contrastive loss (maximize positive, minimize negative)
    loss = -torch.log(torch.sigmoid(pos_score) + 1e-15) - torch.log(1 - torch.sigmoid(neg_score) + 1e-15)
    return loss.mean()

In [49]:
# Define the ModularityLoss module
class ModularityLoss(nn.Module):
    def __init__(self, g):
        super(ModularityLoss, self).__init__()
        self.g = g

    def forward(self, node_embeddings, predicted_communities):
        Q = self.compute_modularity(node_embeddings, predicted_communities)
        loss = 1 - Q  # Minimize 1 - Modularity

        return loss

    def compute_modularity(self, node_embeddings, predicted_communities):
        communities = torch.tensor(predicted_communities)

        # Construct a tensor for community assignment
        community_assignment = torch.zeros((self.g.number_of_nodes(), len(communities)))
        community_assignment.scatter_(1, communities.view(-1, 1), 1)

        # Compute the modularity matrix
        modularity_matrix = self.g.adjacency_matrix().to_dense() - torch.mm(community_assignment, community_assignment.t())

        # Compute the modularity score
        Q = torch.trace(torch.mm(torch.mm(node_embeddings.t(), modularity_matrix), node_embeddings))

        return Q

In [50]:
modularity_loss = ModularityLoss(subgraph)

In [75]:
adj_matrix = subgraph.adjacency_matrix()

FileNotFoundError: Cannot find DGL C++ sparse library at c:\Users\ACT\anaconda3\envs\nocd\Lib\site-packages\dgl\dgl_sparse\dgl_sparse_pytorch_2.2.1.dll

In [74]:
adj_matrix.sparse()

AttributeError: 'tuple' object has no attribute 'sparse'

In [59]:
model.train()
node_feats= model(subgraph, subgraph.ndata['feature'], subgraph.edata['feature'])

node_feats=node_feats.reshape(node_feats.shape[0],node_feats.shape[3])

# Apply K-means clustering to the learned embeddings
kmeans = KMeans(n_clusters=out_feats, random_state=42)
predicted_communities = kmeans.fit_predict(node_feats.detach().numpy())

predicted_communities = predicted_communities.astype(np.int64)

# node_feats=node_feats.to(torch.int64)

# # Compute the modularity-based loss
loss = modularity_loss(node_feats, predicted_communities)

# optimizer.zero_grad()
# loss.backward()
# optimizer.step()

C:\Users\ACT\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


FileNotFoundError: Cannot find DGL C++ sparse library at c:\Users\ACT\anaconda3\envs\nocd\Lib\site-packages\dgl\dgl_sparse\dgl_sparse_pytorch_2.2.1.dll

In [57]:
predicted_communities

array([0, 0, 0, ..., 0, 0, 1])

In [54]:
from tqdm import tqdm

num_epochs=100
# Training loop
for epoch in tqdm(range(num_epochs),desc='processing'):
    model.train()
    node_feats= model(subgraph, subgraph.ndata['feature'], subgraph.edata['feature'])
    
    node_feats=node_feats.reshape(node_feats.shape[0],node_feats.shape[3])
    
    # Apply K-means clustering to the learned embeddings
    kmeans = KMeans(n_clusters=out_feats, random_state=42)
    predicted_communities = kmeans.fit_predict(node_feats.detach().numpy())
    
    node_feats=node_feats.to(torch.int64)

    # Compute the modularity-based loss
    loss = modularity_loss(node_feats, predicted_communities)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

processing:   0%|          | 0/100 [00:00<?, ?it/s]C:\Users\ACT\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
processing:   0%|          | 0/100 [00:01<?, ?it/s]


RuntimeError: scatter(): Expected dtype int64 for index

In [17]:
# After training, you can use the learned embeddings for downstream tasks
model.eval()
with torch.no_grad():
    node_embeddings = model(subgraph, subgraph.ndata['feature'], subgraph.edata['feature'])

In [20]:
node_embeddings=node_embeddings.reshape(10693,2)

In [12]:
node_embeddings=node_embeddings.mean(dim=(1,2))

In [48]:
node_embeddings.shape[0]

10693

In [21]:
import numpy as np
from sklearn.cluster import KMeans

# Assuming 'node_embeddings' contains the learned node embeddings
# It should be a numpy array with shape (num_nodes, embedding_dim)

# Specify the number of communities you want to detect (you may need to tune this)
num_communities = 2

# Apply K-means clustering to the learned embeddings
kmeans = KMeans(n_clusters=num_communities, random_state=42)
community_assignments = kmeans.fit_predict(node_embeddings)

C:\Users\ACT\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [26]:
# Get the unique communities
unique_communities = np.unique(community_assignments)

# Create the community affiliation matrix using one-hot encoding
community_affiliation_matrix = np.eye(len(unique_communities))[community_assignments]


In [33]:
Z_pred=community_affiliation_matrix

In [30]:
df = pd.read_csv('./dataset/ogbn_proteins/raw/node_species.csv.gz', header=None, names=['species_id'])
selected_species=[4932,511145]

df_selected_species = df[df['species_id'].isin(selected_species)]

affiliation_matrix = pd.get_dummies(df_selected_species, columns=['species_id']).groupby(level=0, axis=1).max()

C:\Users\ACT\AppData\Local\Temp\ipykernel_28956\1961254491.py:6: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  affiliation_matrix = pd.get_dummies(df_selected_species, columns=['species_id']).groupby(level=0, axis=1).max()


In [31]:
Z_gt=affiliation_matrix

In [36]:
Z_pred.shape

(10693, 2)

In [38]:
Z_gt.shape

(10693, 2)

In [32]:
def get_comlist(community_matrix):
    transposed_matrix = np.transpose(community_matrix)

    # Initialize an empty list to store tuples (community_id, nodes_list)
    community_nodes_list = []

    # Iterate through rows (communities)
    for community_id, community_row in enumerate(transposed_matrix):
        # Find nodes (columns) where the value is 1
        community_nodes = np.where(community_row == 1)[0].tolist()
        
        # Append a tuple to the list containing community ID and nodes list
        community_nodes_list.append(community_nodes)
    
    return community_nodes_list

In [44]:
Z_gt=Z_gt=affiliation_matrix.astype('int').values

In [45]:
nmi = nocd.metrics.overlapping_nmi(Z_pred, Z_gt)
nmi

0.0011373898786996338

In [46]:
from cdlib import NodeClustering
from cdlib import evaluation
import networkx as nx

G=dgl.to_networkx(subgraph)

coms1=get_comlist(Z_gt)
coms2=get_comlist(Z_pred)

communities1=NodeClustering(coms1,G)
communities2=NodeClustering(coms2,G)

onmi=evaluation.overlapping_normalized_mutual_information_LFK(communities1,communities2)
print(onmi)

MatchingResult(score=0.001146597878475153, std=None)
